In [1]:
import json
import pandas as pd
import requests

In [2]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [3]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [18]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [19]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [19]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [20]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [21]:
access_token = get_access_token(client_key, client_secret)

keeping track with calls
27.02 - we ran keys2 unril 1000 

1.03 - keys2 1001:2002

1.03 - keys1 2002:3003

2.03 - keys1 3003:4004

2.03 - keys2 4004:5005

3.03 - keys1 5005:6006

3.03 - keys2 6006:7007


In [22]:
reposted_videos = []
for username in usernames[6006:7007]:
    reposted_videos.append(get_repost_info(username, access_token))

In [28]:
reposted_videos

[{'cursor': -1, 'has_more': False},
 {},
 {'cursor': -1, 'has_more': False},
 {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 116858,
    'id': 7208734164365888810,
    'username': 'oceanictherapy'},
   {'id': 7206640701969304837,
    'username': 'jeffreyjkieffer',
    'favorites_count': 203961},
   {'username': 'z.tyl', 'favorites_count': 5626, 'id': 7185935878647270702},
   {'favorites_count': 26673,
    'id': 7207662707766512938,
    'username': 'bobbykhor'}]},
 {'has_more': False,
  'reposted_videos': [{'favorites_count': 205022,
    'id': 7416858492016921898,
    'username': 'candaceoshow'}],
  'cursor': -1},
 {'has_more': False,
  'reposted_videos': [{'favorites_count': 5,
    'id': 7214458621466414382,
    'username': 'tkhmz116'}],
  'cursor': -1},
 {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 112,
    'id': 7281026356916358405,
    'username': 'dylandeschamps_'}]},
 {'cursor': 21,
  'has_more': True,
  'reposted_vid

In [24]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [29]:
combined = {}
for i, username in enumerate(usernames[6006:7007]):
    combined[username] = reposted_videos[i]
    
combined 


{'travelwhilebroke': {'cursor': -1, 'has_more': False},
 'freedomoftheroad': {},
 'tian6257mh2': {'cursor': -1, 'has_more': False},
 'kevinscene': {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 116858,
    'id': 7208734164365888810,
    'username': 'oceanictherapy'},
   {'id': 7206640701969304837,
    'username': 'jeffreyjkieffer',
    'favorites_count': 203961},
   {'username': 'z.tyl', 'favorites_count': 5626, 'id': 7185935878647270702},
   {'favorites_count': 26673,
    'id': 7207662707766512938,
    'username': 'bobbykhor'}]},
 'marsustentable1': {'has_more': False,
  'reposted_videos': [{'favorites_count': 205022,
    'id': 7416858492016921898,
    'username': 'candaceoshow'}],
  'cursor': -1},
 'zhangidvla0': {'has_more': False,
  'reposted_videos': [{'favorites_count': 5,
    'id': 7214458621466414382,
    'username': 'tkhmz116'}],
  'cursor': -1},
 'wangjiangyundong': {'cursor': -1,
  'has_more': False,
  'reposted_videos': [{'favorites_count': 11

In [26]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

             username             video_id             creator  favourites
0          kevinscene  7208734164365888810      oceanictherapy      116858
1          kevinscene  7206640701969304837     jeffreyjkieffer      203961
2          kevinscene  7185935878647270702               z.tyl        5626
3          kevinscene  7207662707766512938           bobbykhor       26673
4     marsustentable1  7416858492016921898        candaceoshow      205022
...               ...                  ...                 ...         ...
7752     n1ghtwolf623  7476179209828764971           charl3y14         232
7753     n1ghtwolf623  7476168795334216991        jaime_jane84          36
7754     n1ghtwolf623  7476369416595246379      nelsonbrandi24         302
7755     n1ghtwolf623  7477070543523450158  thedailyreportnews        6715
7756     n1ghtwolf623  7476092452886121770           lalodoe12         339

[7757 rows x 4 columns]


In [27]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)